# IMPORTING THE LIB

In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as mlt
import numpy as np

# Loading the data set

In [2]:
IMGSIZE=224
BATCHSIZE=128
CHANELS=3

In [3]:
data1=tf.keras.preprocessing.image_dataset_from_directory(
    "cancer_img",
    shuffle=True,
    image_size=(IMGSIZE,IMGSIZE),
    batch_size=BATCH_SIZE
)

Found 15000 files belonging to 3 classes.


# Data visualization

In [4]:
class_names = data1.class_names
class_names

['lung_aca', 'lung_n', 'lung_scc']

In [5]:
len(class_names)

3

In [6]:
 for image_batch, label_batch in data1.take(1):
        print(imagebatch.shape)
        print(imagebatch[0].numpy())

(128, 224, 224, 3)
[[[ 85.474495  49.47449  199.04591 ]
  [ 81.86224   43.581635 192.42346 ]
  [ 98.841835  59.897957 202.64285 ]
  ...
  [109.37239   77.4438   222.17351 ]
  [ 87.43351   50.790684 204.71915 ]
  [ 80.928535  37.000008 188.00002 ]]

 [[ 89.        54.       196.98979 ]
  [ 87.42857   52.428574 198.42857 ]
  [ 79.14285   41.142857 188.68878 ]
  ...
  [ 98.5458    65.61721  214.28056 ]
  [ 73.99989   36.999897 194.64275 ]
  [ 73.28056   29.214277 181.63776 ]]

 [[ 74.984695  43.556126 182.31122 ]
  [ 74.571434  40.571426 186.61734 ]
  [ 75.92347   39.92347  191.92348 ]
  ...
  [ 85.85183   52.923244 203.48457 ]
  [ 66.45405   28.45405  190.79073 ]
  [ 66.357124  21.214304 181.30104 ]]

 ...

 [[201.65816  184.82648  238.96936 ]
  [204.23982  184.04593  239.64287 ]
  [205.57147  181.4337   243.648   ]
  ...
  [172.14816  117.3624   245.28589 ]
  [164.9796   110.97462  238.28577 ]
  [170.85718  120.77046  241.81638 ]]

 [[198.6479   169.79062  237.64786 ]
  [213.48477  181.

In [1]:
mlt.figure(figsize=(20,10))
for image_batch, label_batch in data1.take(1):
    for i in range(12):
        ax = mlt.subplot(3,4,i+1)
        mlt.imshow(image_batch[i].numpy().astype('uint8'))
        mlt.title(class_names[label_batch[i]])
        mlt.axis("off")


NameError: name 'mlt' is not defined

# Spliting the data

In [8]:
len(data1)

118

In [9]:
def get_dataset_div_tf(ds,train_split=0.74,val_split=0.13,test_split=0.13,shuffle=True,shuffle_size=1000):
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=5)
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    return train_ds,val_ds,test_ds
    

In [10]:
train_ds,val_ds,test_ds=get_dataset_div_tf(data1)

In [11]:
len(train_ds)

87

In [12]:
len(val_ds)

15

In [13]:
len(test_ds)

16

## using gpu and cpu together

In [14]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

# Data preprocessing

In [15]:
resize_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMG_SIZE,IMG_SIZE),# to convert the supplied image acc. to the model 
    layers.experimental.preprocessing.Rescaling(1.0/255) # to cover the rgb
    
]) 

In [16]:
#for slight variation like rotation,zoom etc
data_augment=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
    
])

# Buliding the model

In [17]:
input_shape = (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS)
n_classes = len(class_names)


model = models.Sequential([
    resize_rescale,
    data_augment,
    
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (128, 224, 224, 3)        0         
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                  

In [19]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

In [ ]:
history=model.fit(
    train_ds,
    epochs=15,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/15
87/87 [==============================] - 684s 7s/step - loss: 0.5077 - accuracy: 0.7624 - val_loss: 0.3927 - val_accuracy: 0.8516
Epoch 2/15
87/87 [==============================] - 569s 7s/step - loss: 0.3137 - accuracy: 0.8729 - val_loss: 0.2813 - val_accuracy: 0.8797
Epoch 3/15
87/87 [==============================] - 577s 7s/step - loss: 0.2603 - accuracy: 0.8949 - val_loss: 0.2525 - val_accuracy: 0.8880
Epoch 4/15
 3/87 [>.............................] - ETA: 9:00 - loss: 0.2117 - accuracy: 0.9010

In [ ]:
scores=model.evaluate(test_ds)

# Plotting the Accuracy and Loss Curves

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
mlt.figure(figsize=(8, 8))
mlt.subplot(1, 2, 1)
mlt.plot(range(15), acc, label='Training Accuracy')
mlt.plot(range(15), val_acc, label='Validation Accuracy')
mlt.legend(loc='lower right')
mlt.title('Training and Validation Accuracy')

mlt.subplot(1, 2, 2)
mlt.plot(range(15), loss, label='Training Loss')
mlt.plot(range(15), val_loss, label='Validation Loss')
mlt.legend(loc='upper right')
mlt.title('Training and Validation Loss')
mlt.show()

# function

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
mlt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = mlt.subplot(3, 3, i + 1)
        mlt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        mlt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        mlt.axis("off")

In [ ]:
print(class_names[2])

In [ ]:
data1.class_names

In [ ]:
model_No='15_epoch_lung'
model.save(f"../models/{model_No}")

OSError: No file or directory found at f"../models/{model_No}"